<br>

# Running ev-MOGA alogorithm for sustainable portfolio selection

3 objectives

<br>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.io import savemat

# import time
# import os

In [ ]:
import evmoga as ev
from evmoga import utilities as evu

import portfolio_selection_user_functions as uf

## Import data to define the optimization problem

Description and other information about the data:

In [ ]:
dataset = "Eurostoxx50"
# dataset = "DowJones"
# dataset = "FTSE100"
# dataset = "NASDAQ100"
# dataset = "SP500"

excel_input_data = "./Datasets/Datasets_Sustainability/" + dataset + ".xlsx"

price_sheet_name = "AssetPrices"
esg_sheet_name = "ESG"

Nind_P = int(10000) # Main population size
Generations = int(1000) # Number of generations
N_div = int(200) # Number of divisions per objective
Nind_GA = int(200) # Auxiliary population size (mutation + crossover)
Nit = int(100) # Number of iterations for storing time evolution data (Nind_A, spent time)

# UB = 0.20  # Upper Bound for actives in portfolio
# LB = 0.01  # Lower Bound for actives in portfolio
UB = 1.00  # Upper Bound for actives in portfolio
LB = 0.00  # Lower Bound for actives in portfolio

v31 = 0 # v31 \in {0, 25000, 37500, 50000, 100000}
v13 =  2.0e-05 # v13 \in {8.0e-06, 1.4e-05, 2.0e-05, 3.0e-05}
v21 = 0

suffix = "" # For the file name
# suffix = "_Ndiv_100" 
# suffix = "_a_priori" 
# suffix = "_UB_01"
# suffix = "_UB_02"

description = 'Mean - Variance - ESG Score (3 objectives) - ' + dataset
objectives = ["Mean Return", "Variance", "ESG Score"]
signs = [-1, +1, -1]

# Options:
save_results = True
mat_folder = "./mat_files/"
mat_filename = mat_folder + "evMOGApy_" + dataset + suffix + "_3obj.mat" 

Daily and mean returns:

In [ ]:
df = pd.read_excel(excel_input_data, sheet_name=price_sheet_name,
                index_col=0, header=None).dropna(axis=1)
df = df.pct_change(axis=0).dropna(axis=0)
# display(df)

returns = df.values.T
mean_r = returns.mean(axis=1)
cov_Mtrx = np.cov(returns)

print(f"Returns shape = {returns.shape}")
print(f"Mean returns shape = {mean_r.shape}")
print(f"Shape of Covariance matrix = {cov_Mtrx.shape}")

ESG score

In [ ]:
df = pd.read_excel(excel_input_data, sheet_name=esg_sheet_name, 
                index_col=0, header=None).dropna(axis=1)
esg = df.iloc[1:, :].values.mean(axis=0)

print(f"ESG Shape = {esg.shape}")
# display(esg)

## Setting-up and running ev-MOGA

In [ ]:
n_obj = len(signs) # Number of objectives
n_var = mean_r.shape[0] # Number of decision variables

eMOGA = {
    
    'description': description,
    'objectives': objectives,

    'objfun': uf.objective_function_3obj,
    'iterationfun': uf.fun_iteration,
    'resultsfun': uf.fun_results,
    'constraintfun': uf.sum1,

    'objfun_dim': n_obj,
    'searchSpace_dim': n_var,
    'searchspaceUB': UB * np.ones(n_var),
    'searchspaceLB': LB * np.ones(n_var),

    'Nind_P': Nind_P,
    'Generations': Generations,

    'Nind_GA': Nind_GA,
    'n_div': [N_div for i in range(n_obj)],

    'param': {
        'signs': signs,
        'esg': esg,
        'ret': returns,
        'mean_r': mean_r,
        'cov_Mtrx': cov_Mtrx,
        },

    'Sigma_Pm_ini': 20,
    'Sigma_Pm_fin': 0.1,

    'Pm': 0.5,

    # ------------------------------
    # User-defined:

    'precision_onoff': True,
    'precision': 0.001,

    'save_results': save_results,
    'mat_file': mat_filename,

    'Nit': Nit
}

if suffix == "_a_priori":

    v1 = np.matrix([1, 0, v13]).T  # Mean return
    v2 = np.matrix([v21, 1, 0]).T  # Variance
    v3 = np.matrix([v31, 0, 1]).T  # ESG risk

    Mp = np.matrix(np.hstack((v1, v2, v3))) # It is required to be a matrix
    Md = evu.dominanceCone(Mp)
    eMOGA['param']['Md'] = Md
    eMOGA['param']['Md_1'] = np.linalg.inv(Md)

    print(f"Matrix of preference directions Mp = \n{Mp}\n")
    print(f"Matrix of directions of the edges of the dominance cones Md = \n{eMOGA['param']['Md']}\n")
    print(f"Matrix of directions of the edges of the dominance cones Md_1 = \n{eMOGA['param']['Md_1']}\n")


eMOGA = uf.generate_P0(eMOGA)

eMOGA = ev.MOGA(eMOGA)

## Plotting results

In [ ]:
if 'Md' in eMOGA['param']:
    # Md = np.matrix(eMOGA['param']['Md']) # Equivalent
    # pfront = np.matrix(eMOGA['coste_A'])
    # pfront = np.transpose(Md * pfront.T)
    # # display(Md, Md.shape)
    # # display(pfront.T, pfront.T.shape)
    # # display(pfront, pfront.shape)

    Md = eMOGA['param']['Md']
    pfront = eMOGA['coste_A'].copy()
    pfront = np.transpose(np.matmul(Md, pfront.T))
    # display(pfront, pfront.shape)
    
    eMOGA['coste_A_optimization'] = eMOGA['coste_A'].copy()
    eMOGA['coste_A'] = pfront.copy()
    # display(eMOGA['coste_A'], eMOGA['coste_A'].shape)

In [ ]:
dot_size_3D = 1.0
opt_dot_size_factor_3D = 3

dot_size_2D = 2.5
opt_dot_size_factor_2D = 8

eMOGA['signs'] = np.array(eMOGA['param']['signs'])
cmap = 'winter'

if 'graph options' not in eMOGA:
    eMOGA['graph options'] = dict()
eMOGA['graph options']['dot_size_2D'] = dot_size_2D
eMOGA['graph options']['obj_ord_3D'] = [1, 2, 0]
eMOGA['graph options']['dot_size_3D'] = dot_size_3D
eMOGA['graph options']['opt_dot_size_factor_3D'] = opt_dot_size_factor_3D
eMOGA['graph options']['3D_view'] = {
    'azimuth': -135,
    'elevation': 20,
    'distance': 2.5,
}

evu.plot_Pareto_Front(eMOGA, plot_optims=True)
evu.plot_2D_projections(eMOGA)
evu.plot_Level_Diagrams(eMOGA, plot_params_LD=False)

## Saving additional information to eMOGA structure

In [ ]:
if eMOGA['save_results']:
    savemat(eMOGA['mat_file'], eMOGA)

## Exploring results

Optimum:

In [ ]:
k = [i for i in range(len(eMOGA['profiles'])) if eMOGA['profiles'][i]['Name'] == 'Optimum']
if k:
	print(f"Optimum:\n")
	j = [i for i in range(len(eMOGA['profiles'][k[0]]['x'])) if eMOGA['profiles'][k[0]]['x'][i] > 0]
	for i in j:
		print(f"x_{i} = {eMOGA['profiles'][k[0]]['x'][i]:.3f} ({df.index[i]})")
	print(f"\nz = {(eMOGA['profiles'][k[0]]['z'])}")
else:
	print("No 'Optimum' profile found")